In [1]:
%%time
# core importations
from caveclient import CAVEclient
import pandas as pd

# dash importations
import dash
from dash import Dash, dcc, html, Input, Output, State, dash_table

# ----------CORE DATA GATHERING FUCNTIONS----------

# defines function to convert nm coordinates into FlyWire-usable
def coordConvert(coords):
    x = coords
    x[0] /= 4
    x[1] /= 4
    x[2] /= 40
    return x

# defines function to get nucleus table entry using root id
def getNucRoot(root_id):
    client = CAVEclient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"pt_root_id": [root_id]},
                                            materialization_version = mat_vers)
    return nuc_df 

def getNucNuc(nucleus_id):
    client = CAVEclient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"id": [nucleus_id]},
                                            materialization_version = mat_vers)
    return nuc_df

# defines function to get synapse table entry using root id
def getSyn(root_id):
    client = CAVEclient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"pre_pt_root_id":[root_id]})
    return len(syn_df)

# defines function to get edit data
def getEdits(root_id):
    client = CAVEclient("flywire_fafb_production")
    changes_dict = client.chunkedgraph.get_change_log(root_id)
    return changes_dict   

# defines function to build dataframe using root id list and 'options' (list of keywords based on checkboxes)
def dfBuilderRoot(root_list,options):
    data_list = []
    column_list = ['root_id']
    if 'nucs' in options:
        nuc = True
        column_list += ['nucleus_id','nucleus_coordinates','volume']
    else:
        nuc = False
    if 'syns' in options:
        syn = True
        column_list += ['synapses']
    else:
        syn = False
    if 'edits' in options:
        edits = True
        column_list += ['mergers','splits','edits']
    else:
        edits = False     
    for i in root_list:
        sub_list = []
        sub_list.append(i) #adds root#
        if nuc == True:
            try:
                nuc_df = getNucRoot(i) #pulls nucleus table#
                sub_list.append(nuc_df.loc[0,'id']) #adds nucleus id #
                sub_list.append(coordConvert(nuc_df.loc[0,'pt_position'])) #adds coordinates#
                sub_list.append(nuc_df.loc[0,'volume']) #adds volume#
            except:
                sub_list += ['n/a','n/a','n/a']
        if syn == True:
            try:
                sub_list.append(getSyn(i)) #adds synapses#
            except:
                sub_list += ['n/a']
        if edits == True:
            try:
                change_dict = getEdits(i) #pulls changelog#
                sub_list.append(change_dict['n_splits']) #adds splits# 
                sub_list.append(change_dict['n_mergers']) #adds mergers#
                sub_list.append((sub_list[-1] + sub_list[-2])) #adds edits#
#                 sub_list.append(change_dict['user_info']) #adds editors, TEMPORARILY DISABLED#
            except:
                sub_list += ['n/a','n/a','n/a']
            
        data_list.append(sub_list) #adds entry to data_list#

    # converts data_list into dataframe
    df = pd.DataFrame(data_list,columns = column_list)
    return df

# defines function to build dataframe using root id list and 'options' (list of keywords based on checkboxes)
def dfBuilderNuc(nuc_list,options):
    data_list = []
    column_list = ['root_id']
    if 'nucs' in options:
        nuc = True
        column_list += ['nucleus_id','nucleus_coordinates','volume']
    else:
        nuc = False
    if 'syns' in options:
        syn = True
        column_list += ['synapses']
    else:
        syn = False
    if 'edits' in options:
        edits = True
        column_list += ['mergers','splits','edits']
    else:
        edits = False     
     
    for i in nuc_list:
        sub_list = []
        nuc_df = getNucNuc(i) #pulls nucleus table#
        root_id = nuc_df.loc[0,'pt_root_id'] #sets root#
        sub_list.append(root_id) #adds root#
        
        if nuc == True:
            try:
                sub_list.append(i) #adds nucleus id #
                sub_list.append(coordConvert(nuc_df.loc[0,'pt_position'])) #adds coordinates#
                sub_list.append(nuc_df.loc[0,'volume']) #adds volume#
            except:
                sub_list += ['n/a','n/a','n/a']
        if syn == True:
            try:
                sub_list.append(getSyn(root_id)) #adds synapses#
            except:
                sub_list += ['n/a']
        if edits == True:
            try:
                change_dict = getEdits(root_id) #pulls changelog#
                sub_list.append(change_dict['n_splits']) #adds splits# 
                sub_list.append(change_dict['n_mergers']) #adds mergers#
                sub_list.append((sub_list[-1] + sub_list[-2])) #adds edits#
#                 sub_list.append(change_dict['user_info']) #adds editors, TEMPORARILY DISABLED#
            except:
                sub_list += ['n/a','n/a','n/a']
            
        data_list.append(sub_list) #adds entry to data_list#

    # converts data_list into dataframe
    df = pd.DataFrame(data_list,columns = column_list)
    return df
   

Wall time: 1.81 s


In [2]:
# DASH APP

app = dash.Dash(__name__)

# defines layout of various app elements (submission field, checkboxes, button, output table)#
app.layout = html.Div([
    html.Div(dcc.Input(  #defines input field#
        id='input_field', 
        type='text', 
        placeholder='ID Number',
    )),
    html.Br(
    ),
    dcc.Dropdown(
        id='query_type',
        options=[
            {'label': 'Root ID', 'value': 'root_query'},
            {'label': 'Nucleus ID', 'value': 'nuc_query'},
        ],
        value='root_query'
    ),
    html.Br(
    ),
    html.Div(dcc.Checklist(  #defines data selection checkboxes#
        id='checkboxes',
        options=[
            {'label': 'Nucleus Info', 'value': 'nucs'},
            {'label': 'Synapse Count', 'value': 'syns'},
            {'label': 'Edits', 'value': 'edits'},
        ],
        labelStyle={'display': 'block'},
        value=['nucs','syns','edits'],
    )),
    html.Br(
    ),
    html.Button(  #defines submission button#
        'Submit', 
        id='submit_button', 
        n_clicks=0,
    ),
    html.Br(
    ),
    html.Div(dash_table.DataTable(  #defines output table#
        id='table', 
        fill_width=False,  #sets column width to fit text instead of expanding to container width#
        export_format="csv",
    ))
])

# defines callback that takes root ids and desired data selection on button click and generates table
@app.callback(
    Output('table','columns'),           #defines first output location as the 'columns' aspect of 'table'#
    Output('table', 'data'),             #defines second output location as the 'data' aspect of 'table'#
    Input('submit_button', 'n_clicks'),  #defines trigger as button press (change in the state of the 'n_clicks' aspect of 'submit_button')# 
    State('query_type', 'value'),         #defines first input state as value of 'query_type'#
    State('input_field', 'value'),       #defines second input state as the value of 'input_field'#
    State('checkboxes','value'),         #defines third input state as the value of 'checkboxes'#
    prevent_initial_call=True,
)
def update_output(n_clicks, query_method, ids, checked):
    id_list = str(ids).split(",")                             #splits 'roots' string into list#
    id_list = [int(x.strip(' ')) for x in id_list]            #strips spaces from root_list entries and converts to integers#
    if query_method == 'root_query':                          #creates dataframe for root id queries#
        df = dfBuilderRoot(id_list, checked)
    elif query_method == 'nuc_query':                         #creates dataframe for nucleus id queries#
        df = dfBuilderNuc(id_list, checked)
    column_list = [{"name": i, "id": i} for i in df.columns]  #creates column list based on dataframe columns#
    df_dict =  df.to_dict('records')                          #converts df to dictionary#
    output_list = [column_list,df_dict]                       #combines df_dict and column_list into output list#
    return output_list                                        #returns list of column names and data values#

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Oct/2021 13:35:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:09] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:09] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:09] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:09] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 13:35:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/